# Scrapping

On doit scrappé : 
- Titre d'emploi, 
- Le nom de la société, 
- La localisation, 
- Type de contrat, 
- Salaire,
- Les critères du candidat: skills, niveau d'études minimum requis, la formation, niveau d'expérience minimum, les langues

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import numpy as np
import pandas as pd
import re

# Indeed France

In [3]:
def scrappingFR(job):
#     browser = webdriver.Chrome(executable_path="chromedriver/chromedriver.exe")

    browser = webdriver.Firefox(executable_path="geckodriver/geckodriver.exe")

    browser.get('https://www.indeed.fr/')
    browser.maximize_window()
    
    #Search name Job
    searchInput = browser.find_element_by_class_name('icl-TextInput-control')
    searchInput.send_keys(job)
    #Search Where
    searchInputWhere = browser.find_element_by_id('text-input-where')
    while len(searchInputWhere.get_attribute('value')) != 0:
        searchInputWhere.send_keys(Keys.BACKSPACE)
    searchInputWhere.send_keys('France')
    #Click search
    clickSearch = browser.find_element_by_class_name('icl-Button')
    clickSearch.click()
    
    #Dict
    getInfo = {'Job Title': [], 'Name Company': [], 'Location': [], 'Work time': [], 'Type of contract': [], 'Salary': [], 'Desired skills': [], 'Minimum level of education required': [], 'Minimum experience level required': [], 'Spoken languages': []}
    nameJob = browser.find_elements_by_class_name('jobsearch-SerpJobCard')
    
    #Scrapping    
    while True:
        
        #Ouvrir descJob
        for i in nameJob:
            i.click()
            time.sleep(2)
            
            try:
                getInfo['Job Title'].append(i.find_element_by_class_name('title').text)
            except NoSuchElementException:
                getInfo['Job Title'].append('No Info')

            try:
                getInfo['Name Company'].append(i.find_element_by_class_name('company').text)
            except NoSuchElementException:
                getInfo['Name Company'].append('No Info')

            try:
                getInfo['Location'].append(i.find_element_by_class_name('location').text)
            except NoSuchElementException:
                getInfo['Location'].append('No Info')

            descJob = i.find_element_by_xpath("//*[@id='vjs-content']").text
            
            #Work time
            matchWorkTime = re.search(r'\bTemps plein\b|\bTemps partiel\b', descJob)
            try:
                getInfo['Work time'].append(matchWorkTime.group())
            except:
                getInfo['Work time'].append('No Info')

            #Contract
            matchContract = re.search(r'\bCDI\b|\bCDD\b|\bApprentissage\b|\bApprentissage, Contrat pro\b|\bStage\b|\bSTAGE\b|\bStage\b|\bIntérim\b|\bFreelance\b|\bIndépendant\b|\bFreelance \/ Indépendant\b', descJob)
            try:
                getInfo['Type of contract'].append(matchContract.group(0))
            except:
                getInfo['Type of contract'].append('No Info')

            #Salary
            matchSalary = re.search(r"\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par jour)\b|\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par mois)\b|\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par an)\b|\b(?:\d[ - ]*?){3,5}(€ par jour)\b|\b(?:\d[ - ]*?){3,5}( € par jour)\b|\b(?:\d[ - ]*?){3,5}(€ par mois)\b|\b(?:\d[ - ]*?){3,5}( € par mois)\b|\b(?:\d[ - ]*?){3,5}( € par an)\b|\b(?:\d[ - ]*?){3,5}(€ par an)\b", descJob)
            try:
                getInfo['Salary'].append(matchSalary.group(0))
            except:
                getInfo['Salary'].append('No Info')

            #Skills
            matchSkills = re.findall(r'\bPython\b|\bpython\b|\bPYTHON\b|\bR\b|\bSQL\b|\bNoSQL\b|\bGIT\b|\bSpark\b|\bspark\b|\bflask\b|\bFlask\b|\bstreamlit\b|\bStreamlit\b|\bDocker\b|\bdocker\b|\bKubernetes\b|\bkubernetes\b|\bReactJS\b|\bMachine Learning\b|\bDeep Learning\b|\bNLP\b|\bVueJS\b|\bAngularJS\b|\bScala\b|\bscala\b|\bPySpark\b|\bPyspark\b|\bPowerBI\b|\bSQLSERVER\b|\bSQLServer\b|\bDataiku\b|\bdataiku\b|\bKeras\b|\bkeras\b|\btensorflow\b|\bTensorFlow\b|\bTensorflow\b|\bNLU\b|\bPytorch\b|\bpytorch\b|\bPyTorch\b|\bScikitLearn\b|\bscikitlearn\b|\bScikitlearn\b|\bScikit-Learn\b|\bScikit-learn\b|\bSAS\b|\bJava\b|\bjava\b|\bScikit learn\b|\bhadoop\b|\bHadoop\b|\bhive\b|\bHive\b|\bML\/DL\b|\bAzure\b|\bAWS\b', descJob)
            matchSkills = list(dict.fromkeys(matchSkills)) #Supprimer les doublons
            try:
                getInfo['Desired skills'].append(matchSkills)
            except:
                getInfo['Desired skills'].append('No Info')
            
            #Education Level
            matchEduLevel = re.search(r"(\bformation supérieure\b|\bBAC\+5\b|\bBac\+5\b|\bbac\+5\b|\bBac \+ 5\b|\bBac \+ 5 \/ M2\b|\bDiplôme ingénieur\b|\bdiplôme ingénieur\b|\bBAC\+4\b|\bBac\+4\b|\bbac\+4\b|\bBac \+ 4\b|\bMaster 2\b|\bmaster 2\b|\bBAC\+3\b|\bBac\+3\b|\bbac\+3\b|\bBac \+ 3\b|\bgrande école d\'ingénieur\b|\bgrande école d\’ingénieur\b|\bBAC\+4\/5\b|\bBac\+4\/5\b|\bbac\+4\/5\b|\bM2\b|\bCursus ingénieur\b|\bcursus ingénieur\b|\bformation Data Science ou IA\b|\bFormation Data Science ou IA\b|\bformation Data Science\b|\bFormation Data Science\b|\buniversité\b|\bUniversité\b|\bUniversitaire\b|\buniversitaire\b)", descJob)
            try:
                getInfo['Minimum level of education required'].append(matchEduLevel.group(0))
            except:
                getInfo['Minimum level of education required'].append('No Info')

            #Experience Level
            matchExpLevel = re.search(r"(\b0 \- 2 ans\b|\b1 an\b|\b1 ou 2 ans\b|\b2 ans\b|\b2\/3 ans\b|\b3 \- 5 ans\b|\b3 ans\b|\b4 ans\b|\b5 ans\b)", descJob)
            try:
                getInfo['Minimum experience level required'].append(matchExpLevel.group(0))
            except:
                getInfo['Minimum experience level required'].append('No Info')

            #Spoken Languages
            matchSpokenLang = re.search(r"(\bAnglais\b|\bChinois\b|\bArabe\b|\bEspagnol\b|\bItalien\b)", descJob)
            try:
                getInfo['Spoken languages'].append(matchSpokenLang.group(0))
            except:
                getInfo['Spoken languages'].append('No Info')
            
#         try:
#             nextPage = browser.find_element_by_css_selector("[aria-label='Suivant']")
#             nextPage.click()
#             time.sleep(3)
#             #Fermer popup
#             time.sleep(2)
#             browser.find_element_by_class_name('popover-x-button-close').click()
#         except:
#             print("No more pages")
                
        job = 'DatascientistFR'
        df = pd.DataFrame({ key:pd.Series(value) for key, value in getInfo.items() })
        df.to_csv(f'{job}.csv', index=False, encoding='utf-8')
        return df
        
scrappingFR('"Data scientist"')

,Job Title,Name Company,Location,Work time,Type of contract,Salary,Desired skills,Minimum level of education required,Minimum experience level required,Spoken languages
0,Alternance - Data Scientist H/F,Malakoff Humanis,Paris 9e (75),No Info,Apprentissage,No Info,"[R, Python]",Bac+5,No Info,No Info
1,Data Scientist (F/H),Novencia,Paris (75),No Info,No Info,No Info,"[Machine Learning, Python, R, SAS, SQL]",No Info,3 ans,No Info
2,Data Scientist,DEGETEL,Boulogne-Billancourt (92),No Info,No Info,No Info,"[R, python, hadoop, spark, scala]",formation supérieure,3 ans,No Info
3,DATA SCIENTIST,Servier,Suresnes (92),No Info,No Info,65 000 € par an,"[R, Python, scala, Hadoop, Spark, AWS]",formation supérieure,5 ans,No Info
4,Data Scientist H/F,AGYLIS,La Défense (92),Temps plein,CDI,40 000 € - 50 000 € par an,"[Machine Learning, Python, ML/DL, Docker, AWS,...",No Info,1 an,No Info
5,Alternant(e) Data Scientist H/F,TF1,Boulogne-Billancourt (92),No Info,Apprentissage,No Info,"[Azure, Python, SQL, Machine Learning]",No Info,No Info,No Info
6,Data Scientist Knime (1-5ans XP),Cyberdian,Paris 13e (75),No Info,CDI,No Info,"[Spark, PySpark, Scikit-learn]",No Info,No Info,No Info
7,Data Scientist H/F,ACTIVUS Group,Paris (75),Temps plein,CDI,25 000 € - 50 000 € par an,"[Python, ML/DL, Machine Learning, Deep Learnin...",Bac+5,5 ans,No Info
8,DATA SCIENTIST- F/H,Ingeniance,La Défense (92),No Info,CDI,No Info,[],Bac+5,No Info,No Info
9,ASSISTANT DATA SCIENTIST,Total,Nanterre (92),No Info,Apprentissage,No Info,[],bac+5,3 ans,No Info


# Indeed US

In [4]:
def scrappingUS(job):
#     browser = webdriver.Chrome(executable_path="chromedriver/chromedriver.exe")

    browser = webdriver.Firefox(executable_path="geckodriver/geckodriver.exe")

    browser.get(f'https://www.indeed.com/jobs?q=%22{job}%22&l=United+States')
    browser.maximize_window()
    
    #Dict
    getInfo = {'Job Title': [], 'Name Company': [], 'Location': [], 'Job type': [], 'Salary': [], 'Desired skills': [], 'Minimum level of education required': [], 'Minimum experience level required': [], 'Spoken languages': []}
    nameJob = browser.find_elements_by_class_name('jobsearch-SerpJobCard')
    
    #Scrapping    
    while True:
        
        #Ouvrir descJob
        for i in nameJob:
            i.click()
            time.sleep(2)
            
            try:
                getInfo['Job Title'].append(i.find_element_by_class_name('title').text)
            except NoSuchElementException:
                getInfo['Job Title'].append('No Info')

            try:
                getInfo['Name Company'].append(i.find_element_by_class_name('company').text)
            except NoSuchElementException:
                getInfo['Name Company'].append('No Info')

            try:
                getInfo['Location'].append(i.find_element_by_class_name('location').text)
            except NoSuchElementException:
                getInfo['Location'].append('No Info')

            descJob = i.find_element_by_xpath("//*[@id='vjs-container']").text
            
            #Job type
            matchJobType = re.search(r'\bFull-time\b|\bPart-time\b|\bRemote\b|\bInternship\b|\bContract\b|\bTemporary\b|\bComission\b', descJob)
            try:
                getInfo['Job type'].append(matchJobType.group())
            except:
                getInfo['Job type'].append('No Info')

            #Salary
            matchSalary = re.search(r"\$(\d){2,3}\,(\d){3}( \- )\$(\d){2,3}\,(\d){3}( a year)|\$(\d){2,3}\,(\d){3}( a year)|\$(\d){2}\.(\d){2}( \- )\$(\d){2}\.(\d){2}( an hour)|\$(\d){2}\.(\d){2}( an hour)", descJob)
            try:
                getInfo['Salary'].append(matchSalary.group(0))
            except:
                getInfo['Salary'].append('No Info')

            #Skills
            matchSkills = re.findall(r'\bPython\b|\bpython\b|\bPYTHON\b|\bR\b|\bSQL\b|\bNoSQL\b|\bGIT\b|\bSpark\b|\bspark\b|\bflask\b|\bFlask\b|\bstreamlit\b|\bStreamlit\b|\bDocker\b|\bdocker\b|\bKubernetes\b|\bkubernetes\b|\bReactJS\b|\bMachine Learning\b|\bDeep Learning\b|\bNLP\b|\bVueJS\b|\bAngularJS\b|\bScala\b|\bscala\b|\bPySpark\b|\bPyspark\b|\bPowerBI\b|\bSQLSERVER\b|\bSQLServer\b|\bDataiku\b|\bdataiku\b|\bKeras\b|\bkeras\b|\btensorflow\b|\bTensorFlow\b|\bTensorflow\b|\bNLU\b|\bPytorch\b|\bpytorch\b|\bPyTorch\b|\bScikitLearn\b|\bscikitlearn\b|\bScikitlearn\b|\bScikit-Learn\b|\bScikit-learn\b|\bSAS\b|\bJava\b|\bjava\b|\bScikit learn\b|\bhadoop\b|\bHadoop\b|\bhive\b|\bHive\b|\bML\/DL\b|\bAzure\b|\bAWS\b', descJob)
            matchSkills = list(dict.fromkeys(matchSkills)) #Supprimer les doublons
            try:
                getInfo['Desired skills'].append(matchSkills)
            except:
                getInfo['Desired skills'].append('No Info')
            
            #Education Level
            matchEduLevel = re.search(r"(\bHigh School\b|\bhigh school|\bCollege\b|\bGraduate\b|\bMaster Degree\b|\b\bUniversity\b|\buniversity\b|\bmaster\b|\bPhD\b|\bAssociate's degree\b|\bBachelor Degree\b|\bAssociate\b|\bBachelor\b|\bBaccalaureate\b|\bbaccalaureate\b|\bBachelor of Science\b)", descJob)
            try:
                getInfo['Minimum level of education required'].append(matchEduLevel.group(0))
            except:
                getInfo['Minimum level of education required'].append('No Info')

            #Experience Level
            matchExpLevel = re.search(r"\b1 year\b|\b1+ year\b|\b1\-2 years\b|\b1\-3\+ years\b|\b2 years\b|\b2\+ years\b|\btwo (2) years\b|\b4 years\b|\b3 years\b|\b4\+ years\b|\b5 years\b|\b5+ years\b|\b10 years\b|\b4-6+ years\b", descJob)
            try:
                getInfo['Minimum experience level required'].append(matchExpLevel.group(0))
            except:
                getInfo['Minimum experience level required'].append('No Info')

            #Spoken Languages
            matchSpokenLang = re.search(r"(\bSpanish\b|\bChinese\b|\bArab\b|\bFrench\b|\bItalian\b)", descJob)
            try:
                getInfo['Spoken languages'].append(matchSpokenLang.group(0))
            except:
                getInfo['Spoken languages'].append('No Info')
            
#         try:
#             nextPage = browser.find_element_by_css_selector("[aria-label='Suivant']")
#             nextPage.click()
#             time.sleep(3)
#             #Fermer popup
#             time.sleep(2)
#             browser.find_element_by_class_name('popover-x-button-close').click()
#         except:
#             print("No more pages")
                
        job = 'DatascientistUS'
        df = pd.DataFrame({ key:pd.Series(value) for key, value in getInfo.items() })
        df.to_csv(f'{job}.csv', index=False, encoding='utf-8')
        return df
        
scrappingUS('Data scientist')

,Job Title,Name Company,Location,Job type,Salary,Desired skills,Minimum level of education required,Minimum experience level required,Spoken languages
0,Data Scientist Entry Level - Pathrise Recruiti...,Pathrise,Remote,Remote,No Info,"[Python, SQL]",No Info,3 years,No Info
1,Data Scientist,Source Enterprises,"New York, NY",Full-time,No Info,"[R, NoSQL]",No Info,No Info,No Info
2,Data Scientist,TrueAccord,Remote,Remote,No Info,"[Python, Tensorflow, AWS, Spark]",No Info,1-3+ years,No Info
3,"Data Scientist, Junior",Booz Allen Hamilton,"Rome, NY 13441",No Info,No Info,"[Python, Docker, Deep Learning]",University,No Info,No Info
4,Full Time Opportunities for Students or Recent...,Microsoft,"Redmond, WA",No Info,No Info,"[SQL, Hadoop, Python, R, SAS, Machine Learning]",No Info,No Info,No Info
5,Junior Data Scientist,PIÑATA SCIENCES,"Boston, MA",No Info,No Info,"[Python, R, Hadoop]",No Info,1-2 years,No Info
6,Data Scientist,Upland Software,Remote,Remote,No Info,"[Machine Learning, R, Python, SQL]",No Info,5 years,No Info
7,Data Scientist,Intelliloan,"Costa Mesa, CA",Remote,No Info,"[SQL, python]",Bachelor,No Info,No Info
8,Data Scientist,Mars,United States,No Info,No Info,"[Python, PySpark, AWS]",Bachelor,4+ years,No Info
9,Data Scientist,Ferguson,"Newport News, VA 23608",No Info,No Info,"[R, Python, Java, Hadoop, Hive, Spark]",master,No Info,No Info


In [5]:
jobs = ['Data scientist', 'Python', 'JS', 'Java','Data scientist', 'Python', 'JS', 'Java']
countrys = ['France','France','France','France', 'USA','USA','USA','USA']
filenames = [str.lower(job).replace(' ','')+str.lower(country)+'.csv' for job, country in zip(jobs,countrys) ]
df = pd.concat([pd.read_csv(filename, sep=';', encoding="utf-8-sig") for filename in filenames])
df

,Poste,Entreprise,Pays,Ville,Type de contrat,Salaire,Description
0,Alternance - Data Scientist H/F,Malakoff Humanis,France,Paris 9e (75),"Apprentissage, Contrat pro",None,Contrat ALTERNANCE Finalité du poste Au sein d...
1,Data Scientist H/F,AGYLIS,France,La Défense (92),"Temps plein, CDI",40 000 € - 50 000 € par an,AGYLIS est un acteur important des Entreprise ...
2,DATA SCIENTIST- F/H,Ingeniance,France,La Défense (92),CDI,None,"Contexte: Vous assurez la production, la conce..."
3,Data Scientist and Innovation Engineer H/F,HIPAY,France,Levallois-Perret (92),None,None,Notre métier HiPay est une Fintech Européenne ...
4,ASSISTANT DATA SCIENTIST,Total,France,Nanterre (92),"Apprentissage, Contrat pro",None,Monter un système de reporting pour la directi...
...,...,...,...,...,...,...,...
740,Principal Software Engineer/Architect,Microsoft,USA,"Redmond, WA",None,None,Are you interested in joining a startup within...
741,Java Developer,Softweb Solutions,USA,"Chicago, IL",None,None,Responsibilities: Write and maintain scalable ...
742,Sr. Software Engineer,"Homecare Homebase, LLC",USA,"Louisville, KY",None,None,Position Summary A Senior Software Engineer wo...
743,Sr Full Stack Engineer,Inabia Solutions and Consulting,USA,"Redmond, WA 98052",None,None,Job Description: Sr Full Stack Engineer Data ...


In [8]:
df2 = df.drop_duplicates(subset=None, keep="first")
df2

,Poste,Entreprise,Pays,Ville,Type de contrat,Salaire,Description
0,Alternance - Data Scientist H/F,Malakoff Humanis,France,Paris 9e (75),"Apprentissage, Contrat pro",None,Contrat ALTERNANCE Finalité du poste Au sein d...
1,Data Scientist H/F,AGYLIS,France,La Défense (92),"Temps plein, CDI",40 000 € - 50 000 € par an,AGYLIS est un acteur important des Entreprise ...
2,DATA SCIENTIST- F/H,Ingeniance,France,La Défense (92),CDI,None,"Contexte: Vous assurez la production, la conce..."
3,Data Scientist and Innovation Engineer H/F,HIPAY,France,Levallois-Perret (92),None,None,Notre métier HiPay est une Fintech Européenne ...
4,ASSISTANT DATA SCIENTIST,Total,France,Nanterre (92),"Apprentissage, Contrat pro",None,Monter un système de reporting pour la directi...
...,...,...,...,...,...,...,...
735,Senior Developer,VirtualMGA,USA,United States,None,None,Virtual MGA is searching for someone who can w...
737,"Senior Backend Engineer, Remote Assistance",Cruise,USA,"San Francisco, CA 94103 (South of Market area)",None,None,"We're Cruise, the self-driving ride-hailing se..."
740,Principal Software Engineer/Architect,Microsoft,USA,"Redmond, WA",None,None,Are you interested in joining a startup within...
742,Sr. Software Engineer,"Homecare Homebase, LLC",USA,"Louisville, KY",None,None,Position Summary A Senior Software Engineer wo...


In [25]:
import re
salairesFrance = ['0 € - 20 000 € par an', '20 000 € - 40 000 € par an', '40 000 € - 60 000 € par an', '60 000 € - 80 000 € par an', '80 000 € - 100 000 € par an']
salaireUS = ['$0 - $20 000 a year', '$20 000 - $40 000 a year', '$40 000 - $60 000 a year', '$60,000 - $80,000 a year', '$80,000 - $100,000 a year']

# créer une autre colonne pour salaire, après faire un regex pour voir les mots derrière les nombres
# si c'est "par mois" tu fais salaire * 12 et tu fait if salaire_par_an < 60000 and : salaire_par_an> 40000: salaire_class = "40 000 - 60 000 € par an"

salairefr = {
    '0 € - 20 000 € par an': [],
    '20 000 € - 40 000 € par an': [],
    '40 000 € - 60 000 € par an': [], 
    '60 000 € - 80 000 € par an': [], 
    '80 000 € - 100 000 € par an': []
}

salaireus = {
    '$0 - $20 000 a year': [], 
    '$20 000 - $40 000 a year': [], 
    '$40 000 - $60 000 a year': [], 
    '$60,000 - $80,000 a year': [], 
    '$80,000 - $100,000 a year': []
}

salaryHourUs = re.search(r"\$(\d){2,3}\.(\d){3}( \- )\$(\d){2,3}\.(\d){3}( an hour)|\$(\d){2}\.(\d){2}( an hour)|(From )\$(\d){2}( an hour)"), df2['Salaire'])
salaryDayUs = re.search(r"\$(\d){2,3}\,(\d){3}( \- )\$(\d){2,3}\,(\d){3}( a day)"), df2['Salaire'])
salaryMonthUs = re.search(r"\$(\d){1}\,(\d){3}( a month)|\$(\d){1}\,(\d){3}( \- )\$(\d){1}\,(\d){3}( a month)", df2['Salaire'])
salaryYearUs = re.search(r"\$(\d){2,3}\,(\d){3}( \- )\$(\d){2,3}\,(\d){3}( a year)|\$(\d){2,3}\,(\d){3}( a year)"), df2['Salaire'])


salaireJourFr = re.search(r"\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par jour)\b|\b(?:\d[ - ]*?){3,5}(€ par jour)\b)|\b(?:\d[ - ]*?){3,5}( € par jour)\b"), df2['Salaire'])
salaireMoisFr = re.search(r"\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par mois)\b|\b(?:\d[ - ]*?){3,5}(€ par mois)\b|\b(?:\d[ - ]*?){3,5}( € par mois)\b"), df2['Salaire']))
salaireAnFr = re.search(r"\b(?:\d[ - ]*?){3,5}( € \- )(?:\d[ - ]*?){3,5}( € par an)\b|\b(?:\d[ - ]*?){3,5}( € par an)\b|\b(?:\d[ - ]*?){3,5}(€ par an)\b"), df2['Salaire'])

# Salaire par heure en année
for i in salaireHourUs:
    if  0 <= i <= 83: # 20 000 / 12 (mois) = 1666,66667 ensuite 1666,66667 / 4 (semaines) = 416,666667 et finir par jour 416,666667 / 5 (jours) = 83,33333
        day = i * 40
        week = day * 4
        month = 12
        year = week * month
        salaireus['$0 - $20 000 a year'].append(i)
    elif 84 <= i <= 167: # 40 000 / 12 (mois) = 3333,33333 ensuite 3333,33333 / 4 (semaines) = 833,333333 et finir par jour 833,333333 / 5 (jours) = 166,66667
        day = i * 40
        week = day * 4
        month = 12
        year = week * month
        salaireus['$20 000 - $40 000 a year'].append(i)
    elif 167 <= i <= 250: # 60 000 / 12 (mois) = 5000 ensuite 5000 / 4 (semaines) = 1250 et finir par jour 1250 / 5 (jours) = 250
        day = i * 40
        week = day * 4
        month = 12
        year = week * month
        salaireus['$40 000 - $60 000 a year'].append(i)
    elif 250 <= i <= : # 60 000 / 12 (mois) = 5000 ensuite 5000 / 4 (semaines) = 1250 et finir par jour 1250 / 5 (jours) = 250
        day = i * 40
        week = day * 4
        month = 12
        year = week * month
        salaireus['$60 000 - $80 000 a year'].append(i)




In [ ]:
université = {
    'france' : ['Licence', 'Master 1', 'Master 2', 'M1', 'M2']
    'usa' : ['Master']
}

formation_superieure = {
    'france' : ['Bac +2', 'Bac +3', 'Bac +4', 'Cursus ingénieur', 'grande école d\'ingénieur, diplôme ingénieur']
    'usa' : ['Bachelor', 'High School', 'Engineer']
}

eduLevel = [université, formation_superieure]